# Installations:

In [ ]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.8 MB/s eta 0:00:00


# Imports:

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os
import torch
import numpy as np

In [ ]:
import argparse
import time
from tqdm import tqdm
import copy as cp
from torch_geometric.data import Data
import torch.nn.functional as F
from torch.utils.data import random_split
from torch_geometric.data import DataLoader, DataListLoader
from torch_geometric.nn import DataParallel
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool, GATConv

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

# set the device to run on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# define the text cleaning function
def clean_text(text):
    # remove URLs and Twitter handles
    text = re.sub(r"http\S+|@\S+", "", text)
    # remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text).lower()
    # tokenize the text into words
    words = nltk.word_tokenize(text)
    # remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # join the words back into a string
    cleaned_text = ' '.join(words)
    return cleaned_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the inputs:

In [ ]:
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
!unzip trainingandtestdata.zip

--2023-05-12 12:31:46--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2023-05-12 12:31:46--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  46.1MB/s    in 1.7s    

2023-05-12 12:31:48 (46.1 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
!wget https://github.com/zfz/twitter_corpus/blob/master/full-corpus.csv


--2023-05-12 13:04:10--  https://github.com/zfz/twitter_corpus/blob/master/full-corpus.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘full-corpus.csv’

full-corpus.csv         [ <=>                ]   2.44M  14.1MB/s    in 0.2s    

2023-05-12 13:04:11 (14.1 MB/s) - ‘full-corpus.csv’ saved [2556711]



In [ ]:
!git clone https://github.com/cardiffnlp/tweeteval/

Cloning into 'tweeteval'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 370 (delta 13), reused 3 (delta 1), pack-reused 354
Receiving objects: 100% (370/370), 8.49 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [ ]:
train_labels = []
test_labels = []
val_labels = []

with open("/content/tweeteval/datasets/sentiment/train_text.txt", "r") as f:
  train_data = f.readlines()

with open("/content/tweeteval/datasets/sentiment/val_text.txt", "r") as f:
  val_data = f.readlines()

with open("/content/tweeteval/datasets/sentiment/test_text.txt", "r") as f:
  test_data = f.readlines()

with open("/content/tweeteval/datasets/sentiment/train_labels.txt", "r") as f:
    for line in f:
        train_labels.append(int(line.strip()))

with open("/content/tweeteval/datasets/sentiment/test_labels.txt", "r") as f:
    for line in f:
        test_labels.append(int(line.strip()))

with open("/content/tweeteval/datasets/sentiment/val_labels.txt", "r") as f:
    for line in f:
        val_labels.append(int(line.strip()))


In [ ]:
print(train_labels[10:20])

[1, 1, 2, 0, 1, 1, 2, 2, 0, 1]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import os
from torch_geometric.data import Data


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def clean_text(text):
    text = re.sub(r"http\S+|@\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text).lower()
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stopwords.words('english')]
    cleaned_text = ' '.join(words)
    return cleaned_text

# read the Sentiment140 data
# dir_path = 'data/Sentiment140/'
graph_list = []


for i in range(98000): #range(len(target_data)):
    sentiment_label = int(train_labels[i])
    text = train_data[i]
    # tweet_id = len(context_graph_dict) + 1

    cleaned_text = clean_text(text)

    inputs = tokenizer(cleaned_text, return_tensors='pt', truncation=True, padding=True).to(device)
    outputs = model(**inputs).last_hidden_state

    num_nodes = outputs.shape[1]  # number of words in the tweet
    node_features = outputs.squeeze(0).detach().cpu().numpy()  # word embeddings
    edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j], dtype=torch.long).t().contiguous()  # fully-connected graph
    data = Data(x=torch.from_numpy(node_features).float(), edge_index=edge_index, y=torch.tensor(sentiment_label))

    graph_list.append(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# dataloader = DataLoader(data_list, batch_size=32, shuffle=True)
# torch.save(dataloader, '/content/drive/MyDrive/retweets_loader2.pth')

# Creating the model:

# Representation learning block:

In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch.nn import Linear, Dropout, ReLU, ModuleList
from torch_geometric.nn import GATConv, TransformerConv, GCNConv, BatchNorm

class MultiHeadGAT(MessagePassing):
    def __init__(self, in_channels, out_channels, heads, dropout=0.5):
        super(MultiHeadGAT, self).__init__(aggr='add')
        self.heads = heads
        self.dropout = dropout
        self.gat_layers = ModuleList()
        for i in range(heads):
            self.gat_layers.append(GATConv(in_channels, out_channels // heads))

        self.linear = Linear(out_channels, out_channels)
        self.activation = ReLU()

    def forward(self, x, edge_index):
        xs = []
        for i in range(self.heads):
            xs.append(self.gat_layers[i](x, edge_index))
        x = torch.cat(xs, dim=1)
        x = self.linear(x)
        x = self.activation(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        return x

class GNNClassifier(torch.nn.Module):
    def __init__(self, output_dim, input_dim=768, hidden_dim=256, dropout=0.5):
        super(GNNClassifier, self).__init__()
        torch.manual_seed(12345)
        self.gat1 = GATConv(input_dim, hidden_dim)
        self.gat2 = GATConv(hidden_dim, hidden_dim)
        self.gat3 = GATConv(hidden_dim, hidden_dim)
        self.conv_mu = GCNConv(hidden_dim, hidden_dim)
        self.lin = torch.nn.Linear(hidden_dim, hidden_dim)
        self.lin2 = torch.nn.Linear(hidden_dim, 3)
        self.bn1 = BatchNorm(hidden_dim)
        self.bn2 = BatchNorm(hidden_dim)
        self.bn3 = BatchNorm(hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gat1(x, edge_index)
        x = F.elu(self.bn1(x))
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.gat2(x, edge_index)
        x = F.elu(self.bn2(x))
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.gat3(x, edge_index)
        x = F.elu(self.bn3(x))
        x = F.dropout(x, p=0.4, training=self.training)
        
        x = self.conv_mu(x, edge_index)

        x = F.dropout(x, p=0.4, training=self.training)
        x = global_mean_pool(x, data.batch)
        # x = self.global_pool(x)
        x = self.lin(x)
        x = self.lin2(x)
        x = F.softmax(x, dim=1)

        return x

In [ ]:
# zzz = model(batch)
print(target_data[1000:1700])

# Training:

In [ ]:
!pip install -q pytorch-ignite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 5.2 MB/s eta 0:00:00


In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
    print(f'Training Epoch {epoch+1}: Loss={loss.item()}')
    return loss

def evaluate(model, device, loader, criterion):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            loss += criterion(out, data.y).item()
            pred = out.max(1)[1]
            correct += pred.eq(data.y).sum().item()
    loss /= len(loader)
    accuracy = 100. * correct / len(loader.dataset)
    print(f'Evaluation: Average Loss={loss:.4f}, Accuracy={accuracy:.2f}%')
    return loss, accuracy

def run_validation(engine, val_loader):
    engine.eval()
    with torch.no_grad():
        val_metrics = {name: 0.0 for name in val_metrics}
        for data in tqdm(val_loader, desc='Validation'):
            data = data.to(device)
            output = model(data.x, data.edge_index)
            loss = criterion(output, data.y)
            for name, metric in val_metrics.items():
                val_metrics[name] += metric(output, data.y).item() * data.num_graphs
        for name in val_metrics:
            val_metrics[name] /= len(val_loader.dataset)
    return val_metrics


def validate_one_epoch(model, validation_loader, criterion, device):
    model.eval()
    validation_loss = 0.0
    with torch.no_grad():
        for batch in validation_loader:
            inputs, targets = batch.to(device), batch.y
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            validation_loss += loss.item() * inputs.size(0)
    validation_loss /= len(validation_loader.dataset)
    print(f'Evaluation: Average Loss={validation_loss:.4f}')
    return validation_loss

In [ ]:
class EarlyStopper:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
from torch_geometric.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping
from ignite.engine import Engine, Events
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNClassifier(3).to(device)
optimizer = Adam(model.parameters(), lr=0.002)
criterion = CrossEntropyLoss()
loss_fn = criterion



random_seed = 100
num_epochs = 100


train_data, test_data = train_test_split(graph_list, test_size=0.3, random_state=random_seed)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=random_seed)


train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)

early_stopper = EarlyStopper(patience=2, min_delta=10)


train_losses = []
val_losses = []
early_stopper = EarlyStopper(patience=3, min_delta=10)


# plt.ion()

train_losses = []
val_losses = []
# fig, ax = plt.subplots()

#     train_loss = 0
#     for batch in train_loader:
#         optimizer.zero_grad()
#         out = model(batch.to(device))
#         loss = criterion(out, batch.y)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item() * len(batch)
#     train_loss /= len(train_data)
#     train_losses.append(train_loss)

#     val_loss = 0
#     for batch in val_loader:
#         out = model(batch.to(device))
#         loss = criterion(out, batch.y)
#         val_loss += loss.item() * len(batch)
#     val_loss /= len(val_data)
#     val_losses.append(val_loss)

#     ax.clear()
#     ax.plot(train_losses, label='Train')
#     ax.plot(val_losses, label='Validation')
#     ax.legend()
#     ax.set_title(f'Epoch {epoch+1}/{num_epochs}')
#     ax.set_xlabel('Epoch')
#     ax.set_ylabel('Loss')
#     plt.show()
    
#     print(f'Epoch {epoch+1}/{num_epochs}: Loss={loss.item()}, Val Loss={val_loss}')
#     if early_stopper.early_stop(val_loss):
#         print('Early stopping criterion met!')
#         break


# plt.ioff()
# plt.show()


for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.to(device))
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

    train_loss = 0
    for batch in train_loader:
        out = model(batch.to(device))
        train_loss += criterion(out, batch.y).item() * 32
    train_loss /= len(train_data)
    train_losses.append(train_loss)

    val_loss = 0
    for batch in val_loader:
        out = model(batch.to(device))
        val_loss += criterion(out, batch.y).item() * 32
    val_loss /= len(val_data)
    val_losses.append(val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}: Loss={loss.item()}, Val Loss={val_loss}')
    if early_stopper.early_stop(val_loss):             
        break

# plt.plot(train_losses, label='Training Loss')
# plt.plot(val_losses, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()
    
    # Evaluate the model


# for epoch in np.arange(num_epochs):
#     train_loss = train(model, device, train_loader, optimizer, criterion, epoch)
#     validation_loss = validate_one_epoch(model, val_loader, criterion, device)
#     if early_stopper.early_stop(validation_loss):             
#         break

In [ ]:
from torch_geometric.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping
from ignite.engine import Engine, Events
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

# Initialize the model and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNNClassifier(num_classes=3).to(device)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = CrossEntropyLoss()
loss_fn = criterion

# create the trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, loss_fn, device=device)
evaluator = create_supervised_evaluator(model, metrics={'accuracy': Accuracy(), 'loss': Loss(loss_fn)}, device=device)
validator = create_supervised_evaluator(model, metrics={'accuracy': Accuracy(), 'loss': Loss(loss_fn)}, device=device)

# define the EarlyStopping handler
early_stopping = EarlyStopping(patience=5, score_function=lambda engine: engine.state.metrics['accuracy'], trainer=trainer)

# attach the EarlyStopping handler to the trainer
trainer.add_event_handler(Events.EPOCH_COMPLETED, early_stopping)

# Load the data into a DataLoader
from sklearn.model_selection import train_test_split

# set the random seed for reproducibility
random_seed = 42
num_epochs = 5

# split the data into train and test subsets
train_data, test_data = train_test_split(graph_list, test_size=0.3, random_state=random_seed)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=random_seed)

# split the train data into train and validation subsets
# train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=random_seed)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# # Train the model
# for epoch in range(num_epochs):
#     for batch in train_loader:
#         optimizer.zero_grad()
#         out = model(batch)
#         loss = criterion(out, batch.y)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {epoch+1}/{num_epochs}: Loss={loss.item()}')
#     # Evaluate the model

def train(engin, batch):
    model.train()
    optimizer.zero_grad()
    out = model(batch.to(device))
    loss = criterion(out, batch.y)
    loss.backward()
    optimizer.step()
    # print(f'Training Epoch {epoch+1}: Loss={loss.item()}')
    return loss.item()
    

def run_validation(engine, batch):
    model.eval()
    with torch.no_grad():
        out = model(batch.to(device))
        loss = criterion(out, batch.y)
        return {'loss': loss.item()}

scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

trainer = Engine(train)
# validator = Engine(run_validation)


for epoch in range(num_epochs):
    trainer.run(train_loader)
    validator.run(val_loader)
    train_loss = trainer.state.output
    val_loss = validator.state.metrics['loss']
    print(f'Epoch {epoch+1}/{num_epochs}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}')
    scheduler.step(val_loss)
    
    # Check if early stopping criterion is met
    if early_stopping is not None:
        if early_stopping.step(val_loss):
            print("Early stopping triggered!")
            break


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def evaluate(model, loader, criterion):
    model.eval()
    eval_loss = 0
    eval_acc = 0
    eval_f1 = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            loss = criterion(out, data.y)
            print(out, data.y)
            eval_loss += loss.item()
            eval_acc += accuracy_score(torch.argmax(out.cpu().detach(), axis=1), data.y.cpu())
            eval_f1 += f1_score(torch.argmax(out.cpu().detach(), axis=1), data.y.cpu(), average='weighted')
    eval_loss /= len(loader)
    eval_acc /= len(loader)
    eval_f1 /= len(loader)
    return eval_loss, eval_acc, eval_f1

In [ ]:
test_loss, test_acc, eval_f1 = evaluate(model, test_loader, criterion)
print(f'test F1 score: {eval_f1:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

In [ ]:
loss = criterion(out, data.y)
print(loss)

In [ ]:
import torch_geometric.utils as utils

train_losses = []
val_losses = []
train_accs = []
val_accs = []
loss_list = []

input_dim, hidden_dim, output_dim = 768, 1000, 240

dataloader = DataLoader(context_graph_list, batch_size=batch_size, shuffle=True)

# Initialize the attentional graph autoencoder model and move it to the device
model = ContextAE(input_dim, hidden_dim, output_dim).to(device)

# Define the optimizer for the VAE
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the VAE model
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0

    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        # edge_index = utils.batched_edge_indices(batch.edge_index, batch.batch).to(device)
        x_hat, z_mean, z_log_std = model(batch.x, batch.edge_index.long(), batch.batch)
        # loss = loss_function(x_hat, z_mean, batch.x, z_log_std)
        # loss = loss_function(x_hat, batch.x, z_mean, z_log_std, alpha)
        loss = F.mse_loss(x_hat, batch.x)
        # print(loss)
        # print(kl_loss(z_mean, z_log_std))
        loss = loss + 0.1 * kl_loss(z_mean, z_log_std)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * batch.num_graphs
    train_loss /= len(context_graph_list)
    train_losses.append(train_loss)
    print(f'Epoch {epoch+1}/{num_epochs}: Loss={loss.item()}')
    loss_list.append(loss.item())

Epoch 1/100: Loss=0.3399708867073059
Epoch 2/100: Loss=0.30594226717948914
Epoch 3/100: Loss=0.27572205662727356
Epoch 4/100: Loss=0.2728796899318695
Epoch 5/100: Loss=0.2628888487815857
Epoch 6/100: Loss=0.24702832102775574
Epoch 7/100: Loss=0.24677079916000366
Epoch 8/100: Loss=0.23996798694133759
Epoch 9/100: Loss=0.25310689210891724
Epoch 10/100: Loss=0.23860761523246765
Epoch 11/100: Loss=0.24289673566818237
Epoch 12/100: Loss=0.2468884289264679
Epoch 13/100: Loss=0.2524784207344055
Epoch 14/100: Loss=0.2519073784351349
Epoch 15/100: Loss=0.2421337068080902
Epoch 16/100: Loss=0.2394917756319046
Epoch 17/100: Loss=0.24272757768630981
Epoch 18/100: Loss=0.24628682434558868
Epoch 19/100: Loss=0.25697118043899536
Epoch 20/100: Loss=0.243623748421669
Epoch 21/100: Loss=0.22602272033691406
Epoch 22/100: Loss=0.24247007071971893
Epoch 23/100: Loss=0.24273967742919922
Epoch 24/100: Loss=0.24658086895942688
Epoch 25/100: Loss=0.23415136337280273
Epoch 26/100: Loss=0.23663316667079926
Epoch

* save the model:

In [ ]:
state_dict = model.state_dict()

# Save the state dictionary to a file
torch.save(state_dict, '/content/drive/MyDrive/retweet_cascade/cont_embedder_model.pt')

* embed the data:

In [ ]:
# Embed the input graphs using the trained VAE
cont_embeddings = []
cont_embedder_model = model

for graph in context_graph_list:
    graph = graph.to(device)
    with torch.no_grad():
        output = model.encoder(graph.x, graph.edge_index.long(), graph.batch)
    cont_embeddings.append(output[0].cpu().numpy())

In [ ]:
cont_embeddings = np.array(cont_embeddings[:,0,:])
np.save('/content/drive/MyDrive/retweet_cascade/cont_embeddings.npy', cont_embeddings)

* Save the model:

embed the data:

* some more imports:

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, VGAE
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import global_mean_pool

from torch_geometric.data import DataLoader
from torch_geometric.datasets import Planetoid

from sklearn.metrics import f1_score

In [ ]:
class GCNSentiment(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, concat=True):
      super(GCNSentiment, self).__init__()

      self.num_features = input_dim
      self.num_classes = output_dim
      self.nhid = hidden_dim
      self.concat = concat

      self.conv1 = GATConv(self.num_features, self.nhid * 2)
      self.conv2 = GATConv(self.nhid * 2, self.nhid * 2)

      self.fc1 = Linear(self.nhid * 2, self.nhid)

      if self.concat:
        self.fc0 = Linear(self.num_features, self.nhid)
        self.fc1 = Linear(self.nhid * 2, self.nhid)
      self.fc2 = Linear(self.nhid, self.num_classes)

    def forward(self, data):
      x, edge_index, batch = data.x, data.edge_index, data.batch

      x = F.selu(self.conv1(x, edge_index))
      x = F.selu(self.conv2(x, edge_index))
      x = F.selu(global_mean_pool(x, batch))
      x = F.selu(self.fc1(x))
      x = F.dropout(x, p=0.5, training=self.training)

      if self.concat:
        news = torch.stack([data.x[(data.batch == idx).nonzero().squeeze()[0]] for idx in range(data.num_graphs)])
        news = F.relu(self.fc0(news))
        x = torch.cat([x, news], dim=1)
        x = F.relu(self.fc1(x))

      x = F.log_softmax(self.fc2(x), dim=-1)
      return x

class CategoricalCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(CategoricalCrossEntropyLoss, self).__init__()

    def forward(self, inputs, targets):
        # inputs shape: (batch_size, num_classes)
        # targets shape: (batch_size,)
        
        # Convert targets to one-hot encoding
        targets = torch.eye(inputs.size(1))[targets].to(inputs.device)
        
        # Calculate loss
        loss = -torch.sum(targets * torch.log_softmax(inputs, dim=1), dim=1)
        
        # Average loss over batch
        loss = torch.mean(loss)
        
        return loss

def one_hot_ce_loss(outputs, targets):
    criterion = nn.CrossEntropyLoss()
    _, labels = torch.max(targets, dim=1)
    return criterion(outputs, labels)

In [ ]:
class ContextEmbedding(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ContextEmbedding, self).__init__()
        torch.manual_seed(12345)
        self.gat1 = GATConv(input_dim, hidden_dim)
        self.gat2 = GATConv(hidden_dim, hidden_dim)
        self.gat3 = GATConv(hidden_dim, hidden_dim)

        self.conv_mu = GCNConv(hidden_dim, output_dim)
        # self.lin = torch.nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, edge_index, batch):
        # x = batch.x
        # edge_index = batch.edge_index

        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.gat3(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.4, training=self.training)

        x = self.conv_mu(x)
        x = F.dropout(x, p=0.4, training=self.training)
        z = global_mean_pool(x, batch)

        return x


class PropagationEmbedding(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, concat=True):
      super(PropagationEmbedding, self).__init__()

      self.num_features = input_dim
      self.num_classes = output_dim
      self.nhid = hidden_dim
      self.concat = concat

      self.conv1 = GATConv(self.num_features, self.nhid * 2)
      self.conv2 = GATConv(self.nhid * 2, self.nhid * 2)

      self.fc1 = Linear(self.nhid * 2, self.nhid)

      if self.concat:
        self.fc0 = Linear(self.num_features, self.nhid)
        self.fc1 = Linear(self.nhid * 2, self.nhid)
      self.fc2 = Linear(self.nhid, self.num_classes)

    def forward(self, x, edge_index, batch):
      # x, edge_index, batch = data.x, data.edge_index, data.batch

      x = F.selu(self.conv1(x, edge_index))
      x = F.selu(self.conv2(x, edge_index))
      x = F.selu(global_mean_pool(x, batch))
      x = F.selu(self.fc1(x))
      x = F.dropout(x, p=0.5, training=self.training)

      if self.concat:
        news = torch.stack([data.x[(data.batch == idx).nonzero().squeeze()[0]] for idx in range(data.num_graphs)])
        news = F.relu(self.fc0(news))
        x = torch.cat([x, news], dim=1)
        x = F.relu(self.fc1(x))

      # x = F.log_softmax(self.fc2(x), dim=-1)
      return x

class AGCNSent(torch.nn.Module):
    def __init__(self, output_dim):
      super(AGCNSent, self).__init__()
      self.propagationEmb = PropagationEmbedding(1, 8, output_dim, concat=True)
      self.contextEmb = ContextEmbedding(768, 1000, output_dim)
      self.fc1 = Linear(output_dim * 2, 2*output_dim)
      self.fc2 = Linear(output_dim * 2, 3)

    def forward(self, data):
      x1 = self.propagationEmb(data.x_prop, data.edge_index_prop, data)
      x2 = self.contextEmb(data.x_cont, data.edge_index_cont, data)
      x = torch.cat([x1, x2], dim=-1)
      x = self.fc1(x)
      x = self.fc2(x)
      x = F.log_softmax(x, dim=-1)

      return x

In [ ]:
print(dataset.propagation_graphs[10])

Data(x=[120, 1], edge_index=[2, 119], y=1, id='522684367647559681')


# Training the model:

In [ ]:
from torch_geometric.data import Batch

def my_collate(batch):
    return Batch.from_data_list(batch)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_dim = 1
hidden_dim = 16
output_dim = 16
batch_size = 16
epoch_num = 200

# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

dataloader = DataLoader(data_list, batch_size=batch_size, shuffle=True)
# dataloader = DataLoader(data_list, batch_size=batch_size, shuffle=True, collate_fn=my_collate)

model = AGCNSent(output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

criterion = CategoricalCrossEntropyLoss().to(device)

model.train()
for epoch in range(epoch_num):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    # for batch in dataloader:
    for batch in data_list:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch).to(device)

        m = nn.Sigmoid()

        loss = nn.CrossEntropyLoss()(out, batch.y)

        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch.num_graphs
        total_correct += (out.argmax(dim=-1) == batch.y).sum().item()
        total_samples += batch.num_graphs

    avg_loss = total_loss / 2316
    avg_acc = total_correct / total_samples

    print("Epoch ", epoch, " loss: ", avg_loss, " acc: ", avg_acc)
    #     loss.backward()
    #     optimizer.step()
    #     total_loss += loss.item()
    # print("Epoch ", epoch, " loss: ", total_loss / epoch_num) # note!


model.eval()
pred_y = []
true_y = []

for batch in dataloader:
    batch = batch.to(device)
    with torch.no_grad():
        out = model(batch)
        pred = out.argmax(dim=-1)
        pred_y.append(pred.cpu().numpy())
        true_y.append(batch.y.cpu().numpy())

pred_y = np.concatenate(pred_y, axis=0)
true_y = np.concatenate(true_y, axis=0)

f1 = f1_score(true_y, pred_y, average='macro')
print("F1 score: ", f1)

# Let's watch the model!

In [ ]:
y_list = np.load('/content/drive/MyDrive/retweet_cascade/y_list.npy')

In [ ]:
prop_embeddings = np.load('/content/drive/MyDrive/retweet_cascade/prop_embeddings.npy')
cont_embeddings = np.load('/content/drive/MyDrive/retweet_cascade/cont_embeddings.npy')

In [ ]:
feature_mat = np.concatenate((prop_embeddings, cont_embeddings), axis=1)